In [28]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import re
import time
import tweepy
import json
import pandas as pd
import datetime

def APITWEET():
    # Declaramos nuestras Twitter API Keys:
    ACCESS_TOKEN = '1230251564616515586-2KqPsCG2mIJp3irRjENgHpCfQUxTUg'
    ACCESS_TOKEN_SECRET = '6PJfMtYGY7w6csiIX9m1S5jFEKNZ3hE9PVkHKeN1S14iM'
    CONSUMER_KEY = 'koO4XqTuWFr5ADGcE8kjIkVoU'
    CONSUMER_SECRET = '3F4sk9qU8zbKBROuLPUUj1uvE2YuhseXPe0ahMQoivg4icN5bL'
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)
    return api
#Desuso
def get_stuff(nombre=None):
    api = APITWEET()
    stuff = tweepy.Cursor(api.user_timeline, screen_name = nombre, include_rts = True)
    return stuff
#Desuso
def get_tweets(stuff, n):
    #for status in stuff.items(n):
        #print(status.created_at, status.author.screen_name, status.text)
        #print(status)
        #return status
    #return stuff.items(n)
    return list(stuff.items(n))
    #return stuff.page()

def FechaTweeter(palabra):
    anio = int(palabra[-4:])
    meses = {
        "Jan":1,
        "Feb":2,
        "Mar":3,
        "Apr":4,
        "May":5,
        "Jun":6,
        "Jul":7,
        "Aug":8,
        "Sep":9,
        "Oct":10,
        "Nov":11,
        "Dec":12
    }
    mes = meses[palabra[4:7]]
    dia = int(palabra[8:10])
    hora = int(palabra[11:13]) 
    minuto = int(palabra[14:16])
    segundo = int(palabra[17:19])
    return datetime.datetime(anio,mes,dia,hora,minuto,segundo) - datetime.timedelta(hours = 4)

def depurarFuenteTweet(palabra):
    salida = palabra.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">','').replace("</a>","")
    salida = salida.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">',"")
    salida = salida.replace('<a href="https://studio.twitter.com" rel="nofollow">',"")
    salida = salida.replace('<a href="https://mobile.twitter.com" rel="nofollow">',"")
    salida = salida.replace('<a href="http://twitter.com" rel="nofollow">',"")
    return salida

def get_tweetConFecha(user, api = APITWEET()):
    return list(api.user_timeline(screen_name = user, count= 10))

def definirDatasetPorCuenta(cuenta):
#lista = get_tweetConFecha("colmedchile")
    lista = get_tweetConFecha(cuenta)
    salida = []
    for i in lista:  #get_tweetConFecha("colmedchile"):
        jsonObject = i._json.copy()
        datos = {
                    "Contenido" : jsonObject["text"], 
                    "IR" : "https://twitter.com/i/web/status/" + jsonObject["id_str"], 
                    "Fecha" : FechaTweeter(jsonObject["created_at"]).strftime("%d/%m/%Y %H:%M:%S"),
                    "Dispositivo" : depurarFuenteTweet(jsonObject["source"]),
                    "Likes" : jsonObject["favorite_count"],
                    "Retweets" : jsonObject["retweet_count"],
                    "Entidad" : jsonObject["user"]["name"],
                    "Hora" : FechaTweeter(jsonObject["created_at"]).strftime("%H:%M:%S"),
                    "Foto": jsonObject["user"]["profile_image_url"].replace("_normal.","."),
                    "FechaAux": FechaTweeter(jsonObject["created_at"])
                }
        salida.append(datos.copy())
    data = pd.DataFrame(salida)
    return data

def datasetFinalTweet():
    cuentas = [
                "colmedchile",
                "ministeriosalud",
                "opsoms",
                "ispch",
                "SuperDeSalud"
                ]
    salida = []
    for i in cuentas:
        salida.append(definirDatasetPorCuenta(i))
    data = pd.concat(salida)
    data = data.sort_values(by=['FechaAux'])
    del data["FechaAux"]
    data.to_csv("Tweet.csv", index=False)
    return data
    

#tweepy.Cursor(api.search, q='#मराठी OR #माझाक्लिक OR #म')
#tweepy.Cursor(api.friends)
#tweepy.Cursor(api.home_timeline)
#tweepy.Cursor(api.search, url)
#tweepy.Cursor(api.friends, user_id=user_id, count=200).items()
#tweepy.Cursor(api.mentions_timeline, user_id=user_id, count=200).items()
#######https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

In [29]:
datasetFinalTweet().head(50)

,Contenido,IR,Fecha,Dispositivo,Likes,Retweets,Entidad,Hora,Foto
9,"Donar sangre es donar vida.\nSi vas a salir, h...",https://twitter.com/i/web/status/1254802486226...,27/04/2020 12:00:00,TweetDeck,1,1,Instituto de Salud Pública de Chile,12:00:00,http://pbs.twimg.com/profile_images/1219288777...
9,RT @ministeriosalud: Subsec @arturozunigaj:\n\...,https://twitter.com/i/web/status/1254811704766...,27/04/2020 12:36:38,Twitter Web App,0,11,Super de Salud Chile,12:36:38,http://pbs.twimg.com/profile_images/9869698658...
8,"RT @ministeriosalud: ""Vamos a informar de form...",https://twitter.com/i/web/status/1254811719349...,27/04/2020 12:36:41,Twitter Web App,0,17,Super de Salud Chile,12:36:41,http://pbs.twimg.com/profile_images/9869698658...
8,Si tienes a un miembro del hogar diagnosticado...,https://twitter.com/i/web/status/1254817584693...,27/04/2020 13:00:00,TweetDeck,4,1,Instituto de Salud Pública de Chile,13:00:00,http://pbs.twimg.com/profile_images/1219288777...
7,"@DanielaRowlands Hola, buenas tardes. Te dejam...",https://twitter.com/i/web/status/1254819628347...,27/04/2020 13:08:07,Twitter Web App,1,0,Super de Salud Chile,13:08:07,http://pbs.twimg.com/profile_images/9869698658...
6,¿Sabías que la @SuperDeSalud estableció normas...,https://twitter.com/i/web/status/1254826130755...,27/04/2020 13:33:57,Twitter Web App,0,0,Super de Salud Chile,13:33:57,http://pbs.twimg.com/profile_images/9869698658...
7,Conoce más sobre esta nueva medida en https://...,https://twitter.com/i/web/status/1254832688776...,27/04/2020 14:00:01,TweetDeck,0,1,Instituto de Salud Pública de Chile,14:00:01,http://pbs.twimg.com/profile_images/1219288777...
5,Súmate a #YoSirvoAMiPaísEnLaEmergencia \n\n¿Er...,https://twitter.com/i/web/status/1254844152950...,27/04/2020 14:45:34,Twitter Web App,0,1,Super de Salud Chile,14:45:34,http://pbs.twimg.com/profile_images/9869698658...
6,"Si estás en aislamiento domiciliario, es muy i...",https://twitter.com/i/web/status/1254847785045...,27/04/2020 15:00:00,TweetDeck,0,1,Instituto de Salud Pública de Chile,15:00:00,http://pbs.twimg.com/profile_images/1219288777...
4,RT @ministeriosalud: #DonemosSangre y #Cuidémo...,https://twitter.com/i/web/status/1254856386782...,27/04/2020 15:34:11,Twitter Web App,0,13,Super de Salud Chile,15:34:11,http://pbs.twimg.com/profile_images/9869698658...


In [7]:
lista = get_tweetConFecha("ministeriosalud")

In [8]:
lista[0]

Status(_api=<tweepy.api.API object at 0x0000028E68D65E48>, _json={'created_at': 'Mon Apr 27 13:18:55 +0000 2020', 'id': 1254761949990027266, 'id_str': '1254761949990027266', 'text': 'RT @SaludOccidente: Nuevo stock de Elementos de Protección Personal (EPP) se distribuye a establecimientos de la Red @SaludOccidente. La en…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'SaludOccidente', 'name': 'Servicio de Salud Occidente', 'id': 627937872, 'id_str': '627937872', 'indices': [3, 18]}, {'screen_name': 'SaludOccidente', 'name': 'Servicio de Salud Occidente', 'id': 627937872, 'id_str': '627937872', 'indices': [117, 132]}], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 153825949, 'id_str': '153825949', 'name': 'Min

In [14]:
for i in datasetFinalTweet()["Dispositivo"]:
    print(i)

Twitter Web App
Twitter for iPhone
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Web App
Twitter Media Studio
Twitter for iPhone
Twitter for iPhone
Twitter for iPhone
Twitter Media Studio
Twitter for iPhone
Twitter for iPhone
Twitter Media Studio
Twitter Web App
Twitter for iPhone
Twitter Web App
TweetDeck
Twitter for iPhone
TweetDeck
Twitter for iPhone
Twitter for iPhone
TweetDeck
TweetDeck
Twitter for iPhone
TweetDeck
Twitter for iPhone
TweetDeck
TweetDeck
TweetDeck
TweetDeck
TweetDeck
TweetDeck
TweetDeck
TweetDeck


In [15]:
len(datasetFinalTweet()["Dispositivo"])

40

In [12]:
#"opsoms",
#                "ispch",
#                "SuperDeSalud"
lista = get_tweetConFecha("opsoms")

In [15]:
lista[1]._json["user"]["profile_image_url"].replace("_normal.",".")

'http://pbs.twimg.com/profile_images/1096094749902028801/ijbQWupW.png'

In [ ]:
http://pbs.twimg.com/profile_images/2284174872         /7df3h38zabcvjylnyfe3_bigger.png
http://pbs.twimg.com/profile_images/1245851467891736576/TayMmMjI_bigger.jpg